In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [5]:
url = 'http://book.interpark.com/display/collectlist.do?_method=BestsellerHourNew201605&bestTp=1&dispNo=028'
req = requests.get(url)
html = req.text
# html

In [6]:
soup = BeautifulSoup(html, 'html.parser')

In [20]:
interp = soup.select_one('.rankBestContentList ')
itb = interp.select('.listItem.singleType')
len(itb)
it = itb[0]

In [9]:
# 순위, 책제목, 저자, 출판사, 가격(정수로)

In [144]:
rank = it.select_one('.rankNumber.digit2').find('span')['class'][1][-1]
rank = int(rank)
rank

1

In [145]:
a =it.select('.rankBtn_ctrl')[0]['class'][1][-1]
a

'1'

In [147]:
a = it.select('.rankBtn_ctrl')[0]['class'][1][-1]
b = it.select('.rankBtn_ctrl')[1]['class'][1][-1]
a, b
a + b

'13'

In [149]:
if len(it.select('.rankBtn_ctrl')) == 1:
    rank = it.select_one('.rankNumber.digit2').find('span')['class'][1][-1]
    rank = int(rank)
else:
    a = it.select('.rankBtn_ctrl')[0]['class'][1][-1]
    b = it.select('.rankBtn_ctrl')[1]['class'][1][-1]
    rank = a + b

In [41]:
# 책제목
title = it.select_one('.itemName').get_text()
title

'트렌드 코리아 2022 '

In [52]:
# 저자
author = it.select_one('.itemMeta').select_one('.author').get_text()
author

'김난도(金蘭都), 전미영, 최지혜, 이향은, 이준영'

In [53]:
# 출판사
company = it.select_one('.itemMeta').select_one('.company').get_text()
company

'미래의창'

In [67]:
# 가격
price = it.select_one('.price').get_text().strip().replace('\n원', '').replace(',', '') # 정수로 바꾸기 위해 원, 콤마 삭제
price = int(price)
price

16200

In [108]:
lines = []
for it in itb:
    if len(it.select('.rankBtn_ctrl')) == 1:
        rank = it.select_one('.rankNumber.digit2').find('span')['class'][1][8:]
        rank = int(rank)
    else:
        a = it.select('.rankBtn_ctrl')[0]['class'][1][8:]
        b = it.select('.rankBtn_ctrl')[1]['class'][1][8:]
        rank = a + b
    title = it.select_one('.itemName').get_text()
    author = it.select_one('.itemMeta').select_one('.author').get_text()
    company = it.select_one('.itemMeta').select_one('.company').get_text()
    price = it.select_one('.price').get_text().strip().replace('\n원', '').replace(',', '')
    price = int(price)
    
    lines.append([rank, title, author, company, price])

In [109]:
df = pd.DataFrame(lines, columns=['랭킹', '타이틀', '저자', '출판사', '가격'])
df

,랭킹,타이틀,저자,출판사,가격
0,1,트렌드 코리아 2022,"김난도(金蘭都), 전미영, 최지혜, 이향은, 이준영",미래의창,16200
1,2,거인의 포트폴리오,강환국,페이지2북스,16200
2,3,2022 현직교사들이 들려주는 면접레시피,"류은진, 양왕경, 이광한, 이지혜, 양왕경, 이광한, 이지혜",미래가치,42300
3,4,거꾸로 읽는 세계사,유시민(柳時敏),돌베개,15750
4,5,아직 오지 않은 날들을 위하여,파스칼 브뤼크네르/이세진 역,인플루엔셜,14400
5,6,작은 별이지만 빛나고 있어(10만 부 기념 한정판 에디션),소윤,북로망스,13500
6,7,투자는 디테일에 있다,김정환,부케이,16200
7,8,웰씽킹(Wealthinking),켈리 최,다산북스,14400
8,8,오십에 읽는 논어,최종엽,유노북스,14400
9,10,소크라테스 익스프레스(20만 부 기념 윈터 에디션),에릭 와이너/김하현 역,어크로스,16200


In [110]:
df.to_html('인터파크당일베스트15(211206).html', index=False)

In [126]:
# 발행일과 쪽수를 구하기 위해 책 상세 페이지로
href = it.select_one('a')['href']
href
inurl = 'http://book.interpark.com'
sub_url = inurl + href
sub_url

'http://book.interpark.com/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=334240809&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [139]:
# 발행일을 숫자 여섯 자리로 만들기
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')
publ1 = sub_soup.select_one('.optionRight_wrap > div.bookInfoBox > ul').get_text().split('\n')[3][5:].split()[0][2:4]
publ2 = sub_soup.select_one('.optionRight_wrap > div.bookInfoBox > ul').get_text().split('\n')[3][5:].split()[1][:2]
publ3 = sub_soup.select_one('.optionRight_wrap > div.bookInfoBox > ul').get_text().split('\n')[3][5:].split()[2][:2]
publ = publ1 + publ2 + publ3

'08'

In [125]:
pag = sub_soup.select_one('.optionRight_wrap > div.bookInfoBox > ul').get_text().split('\n')[4][5:]
# addr = sub_soup.select_one('.txt_adr').get_text()
pag = int(pag)
pag

300

In [142]:
lines = []
for it in itb:
    if len(it.select('.rankBtn_ctrl')) == 1: # 랭킹구하기. 랭킹이 한줄이면 그대로()
        rank = it.select_one('.rankNumber.digit2').find('span')['class'][1][8:]
        rank = int(rank)
    else:
        a = it.select('.rankBtn_ctrl')[0]['class'][1][8:]
        b = it.select('.rankBtn_ctrl')[1]['class'][1][8:]
        rank = a + b
    title = it.select_one('.itemName').get_text()
    author = it.select_one('.itemMeta').select_one('.author').get_text()
    company = it.select_one('.itemMeta').select_one('.company').get_text()
    price = it.select_one('.price').get_text().strip().replace('\n원', '').replace(',', '')
    price = int(price)

    href = it.select_one('a')['href']
    sub_url = inurl + href
    sub_req = requests.get(sub_url)
    sub_html = sub_req.text
    sub_soup = BeautifulSoup(sub_html, 'html.parser')
    publ = sub_soup.select_one('.optionRight_wrap > div.bookInfoBox > ul').get_text().split('\n')[3][5:].split()[0][2:4] + sub_soup.select_one('.optionRight_wrap > div.bookInfoBox > ul').get_text().split('\n')[3][5:].split()[1][:2] + sub_soup.select_one('.optionRight_wrap > div.bookInfoBox > ul').get_text().split('\n')[3][5:].split()[2][:2]
    pag = sub_soup.select_one('.optionRight_wrap > div.bookInfoBox > ul').get_text().split('\n')[4][5:]
    pag = int(pag)
    
    lines.append([rank, title, author, company, price, publ, pag])

In [143]:
df = pd.DataFrame(lines, columns=['랭킹', '타이틀', '저자', '출판사', '가격', '발행일', '쪽수'])
df

,랭킹,타이틀,저자,출판사,가격,발행일,쪽수
0,1,트렌드 코리아 2022,"김난도(金蘭都), 전미영, 최지혜, 이향은, 이준영",미래의창,16200,211006,452
1,2,거인의 포트폴리오,강환국,페이지2북스,16200,211108,404
2,3,2022 현직교사들이 들려주는 면접레시피,"류은진, 양왕경, 이광한, 이지혜, 양왕경, 이광한, 이지혜",미래가치,42300,211112,1104
3,4,거꾸로 읽는 세계사,유시민(柳時敏),돌베개,15750,211029,404
4,5,아직 오지 않은 날들을 위하여,파스칼 브뤼크네르/이세진 역,인플루엔셜,14400,211112,320
5,6,작은 별이지만 빛나고 있어(10만 부 기념 한정판 에디션),소윤,북로망스,13500,210929,274
6,7,투자는 디테일에 있다,김정환,부케이,16200,211210,424
7,8,웰씽킹(Wealthinking),켈리 최,다산북스,14400,211110,316
8,8,오십에 읽는 논어,최종엽,유노북스,14400,211103,300
9,10,소크라테스 익스프레스(20만 부 기념 윈터 에디션),에릭 와이너/김하현 역,어크로스,16200,211126,524


In [130]:
df.to_html('한은지_인터파크_발행일쪽수.html', index=False)